In [ ]:
# Faire la différence entre bert tokens et mots pendant le calcul des mesures de similarité ?

In [32]:
import sys

sys.path.insert(0,'/home/ytaille/pyner/pyner')

from pyner.datasets import BRATDataset

conll_dataset = BRATDataset(
    train=[
        "/home/ytaille/data/resources/conll/brat/eng/train",
    ],
    test="/home/ytaille/data/resources/conll/brat/eng/test",#"path/to/brat/test",    # None for training only, test directory otherwise
    val="/home/ytaille/data/resources/conll/brat/eng/dev", # first 20% doc will be for validation
    seed=42,  # don't shuffle before splitting1
)

n2c2_dataset = BRATDataset(
    train=[
        "/home/ytaille/data/resources/n2c2/brat_files/train",
    ],
    test="/home/ytaille/data/resources/n2c2/brat_files/test",#"path/to/brat/test",    # None for training only, test directory otherwise
    val="/home/ytaille/data/resources/n2c2/brat_files/test", # first 20% doc will be for validation
    seed=42,  # don't shuffle before splitting
)

quaero_dataset = BRATDataset(
    train=[
        "/home/ytaille/data/resources/quaero/corpus/train/MEDLINE",
    ],
    test="/home/ytaille/data/resources/quaero/corpus/test/MEDLINE",#"path/to/brat/test",    # None for training only, test directory otherwise
    val="/home/ytaille/data/resources/quaero/corpus/dev/MEDLINE", # first 20% doc will be for validation
    seed=42,  # don't shuffle before splitting
)

mantra_dataset = BRATDataset(
    train=[
        "/home/ytaille/data/resources/mantra/Mantra-GSC_new_ann/French/EMEA_ec22-cui-best_man",
    ],
    test="/home/ytaille/data/resources/mantra/Mantra-GSC_new_ann/French/Medline_EN_FR_ec22-cui-best_man",#"path/to/brat/test",    # None for training only, test directory otherwise
    val="/home/ytaille/data/resources/mantra/Mantra-GSC_new_ann/French/Medline_EN_FR_ec22-cui-best_man", # first 20% doc will be for validation
    seed=42,  # don't shuffle before splitting
)

NUMBER OF ENGLISH FILES: 7374


In [3]:
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.utils import tokenize

data_dict = {
    "mantra": mantra_dataset,
    "conll": conll_dataset,
    "n2c2": n2c2_dataset,
    "quaero": quaero_dataset,
}

text_dict = {
    k: [list(tokenize(td['text'], deacc=True, lower=True)) for td in d.train_data]
    for k, d in data_dict.items()
}

all_dictionary = Dictionary(sum(text_dict.values(), []))

corpus_dict = {
    k: [all_dictionary.doc2bow(text) for text in d]
    for k, d in text_dict.items()
}

lda_dict = {
    k: LdaModel(d, num_topics=10, id2word=all_dictionary)
    for k, d in corpus_dict.items()
}

/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
import numpy as np

mantra_topics = lda_dict['mantra'].get_topics()
quaero_topics = lda_dict['quaero'].get_topics()

In [11]:
np.dot(mantra_topics, quaero_topics.T).max()

0.0007967418